In [1]:
cd ..

/home/evan/Documents/github/DataStreams


In [2]:
from datastreams.datastream import Streamer

import matplotlib.pyplot as plt
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
endpoint = 'https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph'

In [5]:
ds = Streamer(endpoint)

In [6]:
ds.queryFields

[FieldPath(https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph, Query, ['prices']),
 FieldPath(https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph, Query, ['assetPairs'])]

In [7]:
eth_where = {'timestamp_lt': 1663907363, 'assetPair': "ETH/USD"}
# btc_where = {'timestamp_lt': 1663907363, 'assetPair': "BTC/USD"}

In [8]:
df = ds.runQuery(ds.queryFields[0], where=eth_where, query_size=1000)

FIELD - prices
Search query for these params!: {'timestamp_lt': 1663907363, 'assetPair': 'ETH/USD'}


In [18]:
df

,prices_id,prices_assetPair_id,prices_timestamp,prices_price
0,ETH/USD/0x5f2c8f68,ETH/USD,2020-08-06 23:16:56,39178992539
1,ETH/USD/0x5f2c917d,ETH/USD,2020-08-06 23:25:49,39343000000
2,ETH/USD/0x5f2c919b,ETH/USD,2020-08-06 23:26:19,39337482558
3,ETH/USD/0x5f2c9d07,ETH/USD,2020-08-07 00:15:03,39537247267
4,ETH/USD/0x5f2c9d15,ETH/USD,2020-08-07 00:15:17,39451623633
...,...,...,...,...
995,ETH/USD/0x5f7c7908,ETH/USD,2020-10-06 14:02:48,34515000000
996,ETH/USD/0x5f7c7af1,ETH/USD,2020-10-06 14:10:57,34619450792
997,ETH/USD/0x5f7c82a4,ETH/USD,2020-10-06 14:43:48,34530000000
998,ETH/USD/0x5f7c870a,ETH/USD,2020-10-06 15:02:34,34295000000
